In [ ]:
! gdown --id 1BGbi1w1UyWX7q9kfiJg4GC_wvBql1DJi /content/drive/MyDrive/CL_Cup_IT_Data_Scince_секция_кейс_VK_датасет.zip

usage: gdown
       [-h]
       [-V]
       [-O OUTPUT]
       [-q]
       [--fuzzy]
       [--id]
       [--proxy PROXY]
       [--speed SPEED]
       [--no-cookies]
       [--no-check-certificate]
       [--continue]
       [--folder]
       [--remaining-ok]
       url_or_id
gdown: error: unrecognized arguments: /content/drive/MyDrive/CL_Cup_IT_Data_Scince_секция_кейс_VK_датасет.zip


In [ ]:
!unzip /content/drive/MyDrive/CL_Cup_IT_Data_Scince_секция_кейс_VK_датасет.zip

Archive:  /content/drive/MyDrive/CL_Cup_IT_Data_Scince_секция_кейс_VK_датасет.zip
  inflating: readme.txt              
  inflating: ranking_train.jsonl     
  inflating: ranking_test.jsonl      


# Обработка test

In [ ]:
import json
import re 
from html import unescape
import pandas as pd

data = []
# Создаем словарь, в котором ключи - это регулярные выражения для поиска смайликов,
# а значения - это строки замены, которые будут использоваться для замены найденных смайликов
smileys = {
    r"[;:]-?[)D\]]": " SMILE ",  # положительные смайлики (;-), ;-D, :-), :-D, :), : D)
    r"[;:]-?\(" : " SAD "  # грустные смайлики
}
with open('/content/ranking_test.jsonl', 'r', encoding='ascii') as f:
    for line in f:
        json_data = json.loads(line.strip())
        post_text = unescape(json_data['text'])
        post_text = post_text.replace('\'', '').replace('\n', '').replace('\t', '')
        post_text = re.sub(r'[^\x00-\x7F]+', '', post_text)
        comments = json_data['comments']
        for comment in comments:
            comment_text = unescape(comment['text'])
            comment_text = re.sub("'", '"', comment_text) #заменяю ' на ", чтобы убрать экранирование \'
            comment_text = comment_text.replace('"', "'") #обратная замена
            for pattern, replacement in smileys.items():
                comment_text = re.sub(pattern, replacement, comment_text)    
            mail_regex = re.compile(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b') # регулярное выражение для поиска всех почт
            comment_text = mail_regex.sub("mail", comment_text)
            # Регулярное выражение для поиска слов, содержащих "mail"
            mail_regex = re.compile(r"\b\w*[-]*mail[-]*\w*\b")
            comment_text = mail_regex.sub("mail", comment_text)
            # Используем регулярное выражение для поиска всех ссылок в тексте
            urls = re.findall(r'(http\S+?)(?=(http|\s|$))', comment_text) 
            # Используем цикл, чтобы заменить каждую ссылку в тексте на эту же ссылку с пробелами по обе стороны
            for url in urls:
                comment_text = comment_text.replace(url[0], f' {url[0]} ')
            comment_text = re.sub(r'[^\x00-\x7F]+', '', comment_text) #убирает непонятные символы unicode 
            comment_text = re.sub(r'\s+', ' ',  comment_text.strip()) #убирает все лишние пробелы
            comment_text = re.sub(r'(https?://\S+|www\.\S+)', 'URL', comment_text) #заменяет ссылки http,https, www на URL
            link_regex = re.compile(r"\[\d+\]")
            comment_text = link_regex.sub("link", comment_text)
            #comment_text = re.sub(r'[^\w\s]|_', '', comment_text) # очистка текста от всех символов, которые не являются алфавитно-цифровыми, лишних пробелов и знаков препинания
            score = comment.get('score')
            data.append({'post_text': post_text, 'comment': comment_text , 'score': score})

df_test = pd.DataFrame(data)

# Обработка train

In [ ]:
import json
import re 
from html import unescape
import pandas as pd

data = []
# Создаем словарь, в котором ключи - это регулярные выражения для поиска смайликов,
# а значения - это строки замены, которые будут использоваться для замены найденных смайликов
smileys = {
    r"[;:]-?[)D\]]": " SMILE ",  # положительные смайлики (;-), ;-D, :-), :-D, :), : D)
    r"[;:]-?\(" : " SAD "  # грустные смайлики
}
with open('/content/ranking_train.jsonl', 'r', encoding='ascii') as f:
    for line in f:
        json_data = json.loads(line.strip())
        post_text = unescape(json_data['text'])
        post_text = post_text.replace('\'', '').replace('\n', '').replace('\t', '')
        post_text = re.sub(r'[^\x00-\x7F]+', '', post_text)
        comments = json_data['comments']
        for comment in comments:
            comment_text = unescape(comment['text'])
            comment_text = re.sub("'", '"', comment_text) #заменяю ' на ", чтобы убрать экранирование \'
            comment_text = comment_text.replace('"', "'") #обратная замена
            for pattern, replacement in smileys.items():
                comment_text = re.sub(pattern, replacement, comment_text)    
            mail_regex = re.compile(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b') # регулярное выражение для поиска всех почт
            comment_text = mail_regex.sub("mail", comment_text)
            # Регулярное выражение для поиска слов, содержащих "mail"
            mail_regex = re.compile(r"\b\w*[-]*mail[-]*\w*\b")
            comment_text = mail_regex.sub("mail", comment_text)
            # Используем регулярное выражение для поиска всех ссылок в тексте
            urls = re.findall(r'(http\S+?)(?=(http|\s|$))', comment_text) 
            # Используем цикл, чтобы заменить каждую ссылку в тексте на эту же ссылку с пробелами по обе стороны
            for url in urls:
                comment_text = comment_text.replace(url[0], f' {url[0]} ')
            comment_text = re.sub(r'[^\x00-\x7F]+', '', comment_text) #убирает непонятные символы unicode 
            comment_text = re.sub(r'\s+', ' ',  comment_text.strip()) #убирает все лишние пробелы
            comment_text = re.sub(r'(https?://\S+|www\.\S+)', 'URL', comment_text) #заменяет ссылки http,https, www на URL
            link_regex = re.compile(r"\[\d+\]")
            comment_text = link_regex.sub("link", comment_text)
            #comment_text = re.sub(r'[^\w\s]|_', '', comment_text) # очистка текста от всех символов, которые не являются алфавитно-цифровыми, лишних пробелов и знаков препинания
            score = comment.get('score')
            data.append({'post_text': post_text, 'comment': comment_text , 'score': score})

df_train = pd.DataFrame(data)

In [ ]:
df_train

,post_text,comment,score
0,"iOS 8.0.1 released, broken on iPhone 6 models,...",I'm still waiting for them to stabilize wifi o...,None
1,"iOS 8.0.1 released, broken on iPhone 6 models,...","For those who upgraded, no need to do a restor...",None
2,"iOS 8.0.1 released, broken on iPhone 6 models,...",Upgraded shortly after it was released and suf...,None
3,"iOS 8.0.1 released, broken on iPhone 6 models,...",I think they were under a lot of pressure on t...,None
4,"iOS 8.0.1 released, broken on iPhone 6 models,...",Fix for those who already updated: URL,None
...,...,...,...
70015,Why does Gmail hate my domain?,I send a LOT of mail each month (mail newslett...,None
70016,Why does Gmail hate my domain?,I hit a similar problems when sending automate...,None
70017,Why does Gmail hate my domain?,That's all a bit presumptive and inflammatory ...,None
70018,Why does Gmail hate my domain?,If the domain is bitbin.de and the mail server...,None


In [ ]:
df_count_comments = df_train.groupby('post_text').agg({'comment': 'count'}).reset_index()

In [ ]:
df_count_comments

,post_text,comment
0,,15
1,"An empirical comparison of C, C++, Java,...",5
2,Kids born later in the year more likely to ...,5
3,Why Arent Android Developers Flocking To Th...,5
4,25 Innovations will kill startups profitabi...,5
...,...,...
87650,z/OMG,5
87651,"zBase A high-performance, elastic, distribute...",5
87652,~ is being removed from Rust,5
87653,~/.osx a collection of sensible defaults for ...,5


In [ ]:
df_count_comments[df_count_comments['comment'] > 5]

,post_text,comment
0,,15
838,-2000 Lines Of Code,10
839,-2000 lines of code,10
1158,177 Days of GitHub,10
2081,"A Brief, Incomplete, and Mostly Wrong History ...",10
...,...,...
87023,"Youre a developer, so why do you work for some...",10
87026,"Youre in the front of a Linux computer, you ne...",10
87048,Youtube is down,10
87195,ZeroRPC,10


# Предобрабокта 

In [ ]:
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
from nltk.stem import PorterStemmer, WordNetLemmatizer

# загружаем стоп-слова
stop_words = set(stopwords.words('english'))

# инициализируем стеммер и лемматизатор
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

# токенизируем и очищаем текст
def preprocess_text(text):
    # токенизация
    tokens = word_tokenize(text.lower())
    
    # удаление стоп-слов и пунктуации
    tokens = [token for token in tokens if token not in stop_words and token.isalpha()]
    
    # стемминг
    stemmed_tokens = [stemmer.stem(token) for token in tokens]
    
    # лемматизация
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
    return stemmed_tokens, lemmatized_tokens

# Для train 19 минут примерно

In [ ]:
for d in data:
    # токенизация, стемминг, лемматизация для 'post_text'
    stemmed_tokens,lemmatized_tokens = preprocess_text(d['post_text'])
    d['post_stemmed_tokens'] = ' '.join(stemmed_tokens)
    d['post_lemmatized_tokens'] = ' '.join(lemmatized_tokens)

     # токенизация, стемминг, лемматизация для 'comment'
    stemmed_tokens, lemmatized_tokens = preprocess_text(d['comment'])
    d['comment_stemmed_tokens'] = ' '.join(stemmed_tokens)
    d['comment_lemmatized_tokens'] = ' '.join(lemmatized_tokens)

In [ ]:
json_data = json.dumps(data)

In [ ]:
with open('train_new.json', 'w') as file:
    file.write(json_data)

In [ ]:
df_train = pd.DataFrame(data)

In [ ]:
df_train

,post_text,comment,score,post_stemmed_tokens,post_lemmatized_tokens,comment_stemmed_tokens,comment_lemmatized_tokens
0,How many summer Y Combinator fundees decided n...,Going back to school is not identical with giv...,0,mani summer combin funde decid continu startup...,many summer combinator fundees decided continu...,go back school ident give founder go back scho...,going back school identical giving founder go ...
1,How many summer Y Combinator fundees decided n...,There will invariably be those who don't see t...,1,mani summer combin funde decid continu startup...,many summer combinator fundees decided continu...,invari see success set fall back origin path f...,invariably see success set fall back original ...
2,How many summer Y Combinator fundees decided n...,For me school is a way to be connected to what...,2,mani summer combin funde decid continu startup...,many summer combinator fundees decided continu...,school way connect go world enter school think...,school way connected going world entered schoo...
3,How many summer Y Combinator fundees decided n...,I guess it really depends on how hungry you ar...,3,mani summer combin funde decid continu startup...,many summer combinator fundees decided continu...,guess realli depend hungri much believ product...,guess really depends hungry much believe produ...
4,How many summer Y Combinator fundees decided n...,I know pollground decided to go back to school...,4,mani summer combin funde decid continu startup...,many summer combinator fundees decided continu...,know pollground decid go back school get combi...,know pollground decided go back school getting...
...,...,...,...,...,...,...,...
440530,Pay your rent with a Credit or Debit card. No ...,Most major banks offer a service called 'bill ...,0,pay rent credit debit card landlord signup requir,pay rent credit debit card landlord signup req...,major bank offer servic call pay free directli...,major bank offer service called pay free direc...
440531,Pay your rent with a Credit or Debit card. No ...,"It costs 3.25%, or $74.25 for the example of $...",1,pay rent credit debit card landlord signup requir,pay rent credit debit card landlord signup req...,cost exampl look rack point best pump payment ...,cost example looking rack point best pumping p...
440532,Pay your rent with a Credit or Debit card. No ...,As many other comments have pointed out almost...,2,pay rent credit debit card landlord signup requir,pay rent credit debit card landlord signup req...,mani comment point almost everyon debit card a...,many comment pointed almost everyone debit car...
440533,Pay your rent with a Credit or Debit card. No ...,My apartment building uses Yapstone's RentPaym...,3,pay rent credit debit card landlord signup requir,pay rent credit debit card landlord signup req...,apart build use yapston charg flat fee use cre...,apartment building us yapstone charge flat fee...


# Создаем сгруппированный по score 0 датафрейм

In [30]:
new_df = df_train[['post_lemmatized_tokens', 'comment_lemmatized_tokens', 'score']]


In [ ]:
new_df

,post_lemmatized_tokens,comment_lemmatized_tokens,score
0,many summer combinator fundees decided continu...,going back school identical giving founder go ...,0
1,many summer combinator fundees decided continu...,invariably see success set fall back original ...,1
2,many summer combinator fundees decided continu...,school way connected going world entered schoo...,2
3,many summer combinator fundees decided continu...,guess really depends hungry much believe produ...,3
4,many summer combinator fundees decided continu...,know pollground decided go back school getting...,4
...,...,...,...
440530,pay rent credit debit card landlord signup req...,major bank offer service called pay free direc...,0
440531,pay rent credit debit card landlord signup req...,cost example looking rack point best pumping p...,1
440532,pay rent credit debit card landlord signup req...,many comment pointed almost everyone debit car...,2
440533,pay rent credit debit card landlord signup req...,apartment building us yapstone charge flat fee...,3


In [2]:
grouped_df = new_df[new_df['score'] == 0]

NameError: ignored

# Популярные слова

In [41]:
from collections import Counter

# Объединяем текстовые данные в одну строку
all_text = ' '.join(grouped_df['post_lemmatized_tokens']) + ' ' + ' '.join(grouped_df['comment_lemmatized_tokens'])

# Разделяем строку на слова
words = all_text.split()

# Считаем частоту каждого слова и сохраняем результаты в словаре
word_freq = Counter(words)

# Получаем список кортежей (слово, частота) отсортированных по убыванию частоты
sorted_word_freq = sorted(word_freq.items(), key=lambda x: x[1], reverse=True)

# Выводим первые 10 самых часто встречающихся слов
for word, freq in sorted_word_freq[:100]:
    print(f"{word}: {freq}")


like: 159136
would: 140939
url: 140377
people: 128312
one: 123870
get: 115971
time: 100709
make: 96624
thing: 90414
think: 90128
hn: 89750
work: 89355
really: 80534
use: 79206
good: 74248
way: 74140
could: 70012
ask: 69914
much: 69893
want: 68683
know: 66374
even: 66190
need: 65776
see: 63678
year: 62926
something: 60840
also: 59847
new: 59078
company: 56236
lot: 54646
go: 52843
google: 51586
problem: 50924
many: 47773
code: 46225
idea: 45472
using: 45470
first: 44995
going: 44978
say: 44652
take: 44372
better: 44055
user: 43368
look: 43224
still: 42987
well: 42663
great: 40897
find: 40684
startup: 40633
day: 40393
article: 40055
site: 39748
point: 38198
job: 37116
right: 37102
u: 36807
actually: 36666
data: 36481
system: 36431
project: 35600
web: 34780
might: 34453
seems: 34394
app: 34267
language: 34202
business: 33613
someone: 33602
money: 33050
show: 32460
back: 32367
used: 32362
ca: 32143
without: 31315
every: 31012
mail: 30942
probably: 30870
never: 30554
page: 30185
read: 29940


In [1]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

wordcloud = WordCloud(width=800, height=400, background_color="white", max_words=50).generate(all_text)

# Отображаем график
plt.figure(figsize=(12, 8))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()


NameError: ignored